# Making Zarr data from NetCDF files

- Funding: Interagency Implementation and Advanced Concepts Team [IMPACT](https://earthdata.nasa.gov/esds/impact) for the Earth Science Data Systems (ESDS) program and AWS Public Dataset Program
- Software developed during [OceanHackWeek 2020](https://github.com/oceanhackweek) 
  
### Credits: Tutorial development
* [Dr. Chelle Gentemann](mailto:gentemann@faralloninstitute.org) -  [Twitter](https://twitter.com/ChelleGentemann)   - Farallon Institute
* [Patrick Gray](mailto:patrick.c.gray@duke.edu) - [Twitter](https://twitter.com/clifgray) - Duke University
* [Phoebe Hudson](mailto:pahdsn@outlook.com) - [Twitter](https://twitter.com/pahdsn) - University of Southampton

## Why data format matters
- NetCDF sprinkles metadata throughout files, making them slow to access and read data
- Zarr consolidates the metadata, making them FAST for access and reading


In [2]:
# filter some warning messages
import warnings 
warnings.filterwarnings("ignore") 

#libraries
import datetime as dt
import xarray as xr
import fsspec
import s3fs
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# make datasets display nicely
xr.set_options(display_style="html")  
import os.path
from os import path

#magic fncts #put static images of your plot embedded in the notebook
%matplotlib inline  
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 


In [6]:
def get_geo_data(sat,lyr,idyjl):
    # arguments
    # sat   goes-east,goes-west,himawari
    # lyr   year
    # idyjl day of year
    
    ds,iexist=[],False
    
    d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl)
    fs = s3fs.S3FileSystem(anon=True) #connect to s3 bucket!

    #create strings for the year and julian day
    imon,idym=d.month,d.day
    syr,sjdy,smon,sdym = str(lyr).zfill(4),str(idyjl).zfill(3),str(imon).zfill(2),str(idym).zfill(2)
    
    #use glob to list all the files in the directory
    if sat=='goes-east':
        file_location,var = fs.glob('s3://noaa-goes16/ABI-L2-SSTF/'+syr+'/'+sjdy+'/*/*.nc'),'SST'
    if sat=='goes-west':
        file_location,var = fs.glob('s3://noaa-goes17/ABI-L2-SSTF/'+syr+'/'+sjdy+'/*/*.nc'),'SST'
    if sat=='himawari':
        file_location,var = fs.glob('s3://noaa-himawari8/AHI-L2-FLDK-SST/'+syr+'/'+smon+'/'+sdym+'/*/*L2P*.nc'),'sea_surface_temperature'
    print(file_location)
    #make a list of links to the file keys
    if len(file_location)<1:
        return file_ob
    file_ob = [fs.open(file) for file in file_location]        #open connection to files
    
    print('all files open')
    #open all the day's data
    with xr.open_mfdataset(file_ob[0:2], combine='nested',concat_dim='time') as ds: #note file is super messed up formatting
        iexist = True  #file exists

        #clean up coordinates which are a MESS in GOES
        #rename one of the coordinates that doesn't match a dim & should
        if not sat=='himawari':
            ds = ds.rename({'t':'time'})
            ds = ds.reset_coords()
            #clean up variables that should be attributes
            varlist = ['retrieval_solar_zenith_angle', 'day_solar_zenith_angle', 'y_image', 'retrieval_local_zenith_angle', 'x_image', 'night_solar_zenith_angle', 'quantitative_local_zenith_angle',
               'SST_night_only_emissive_wavelength', 'SST_night_only_emissive_band_id', 'SST_day_night_emissive_band_ids', 'SST_day_night_emissive_wavelengths']
            for var in varlist:
                try : 
                    ds[var]
                except KeyError: 
                    pass
                else: 
                    ds.attrs[var]=str(ds.variables[var].attrs['long_name']) + ' (' + str(ds.variables[var].attrs['units']) + ') = ' + str(ds.variables[var].values)
                    ds = ds.drop(var, dim=None)
        else:
            ds = ds.rename({'ni':'x','nj':'y'})    
    
    #put in to Celsius
    #ds[var] -= 273.15   #nice python shortcut to +- from itself a-=273.15 is the same as a=a-273.15
    #ds[var].attrs['units'] = '$^\circ$C'
   
    return ds, iexist


## Open GOES-16 (East Coast) Data
- Careful of what you ask for.... each day is about 3 min to access

In [9]:
%%time
lyr = 2020

#satlist = ['goes-east','goes-west','himawari']
satlist = ['himawari']

for sat in satlist:

    init = 0  #reset new data store

    for idyjl in range(210,222,1): #180:201 #6/28/2020-7/18/2020

        print('starting ', idyjl)

        ds,iexist = get_geo_data(sat,lyr,idyjl)
        
        if not iexist:
            continue

        print('writing zarr store')

        if init == 0:
            ds.to_zarr(sat) #, mode='w'
            init = 1
        else:
            ds.to_zarr(sat,append_dim='time')


starting  210
['noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0000/20200729000000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0100/20200729010000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0200/20200729020000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0300/20200729030000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0400/20200729040000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0500/20200729050000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0600/20200729060000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0700/20200729070000-STAR-L2P_GHRSST-SSTsubskin

ValueError: path '' contains a group

In [8]:
%%time 
lyr = 2020

#satlist = ['goes-east','goes-west','himawari']
sat = 'himawari'
idyjl = 210

ds,iexist = get_geo_data(sat,lyr,idyjl)
ds

['noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0000/20200729000000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0100/20200729010000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0200/20200729020000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0300/20200729030000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0400/20200729040000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0500/20200729050000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0600/20200729060000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0700/20200729070000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO

<xarray.Dataset>
Dimensions:                       (time: 2, x: 5500, y: 5500)
Coordinates:
  * x                             (x) float32 -0.15366402 ... 0.15366402
  * y                             (y) float32 0.15366402 ... -0.15366402
  * time                          (time) object 2020-07-29 00:00:00 2020-07-2...
Data variables:
    sst_dtime                     (time, y, x) timedelta64[ns] dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    satellite_zenith_angle        (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    sea_surface_temperature       (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    brightness_temperature_08um6  (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    brightness_temperature_10um4  (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    brightness_temperature_11um2  (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    brightness_temperature_12um3  (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    sses_bias                     (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    sses_standard_deviation       (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    dt_analysis                   (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    wind_speed                    (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    l2p_flags                     (time, y, x) int16 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    quality_level                 (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    geostationary                 (time) int32 -2147483647 -2147483647
Attributes:
    CFAC:                       [20466275]
    COFF:                       [2750.5]
    Coefficient_for_Sd:         [1.73712226e+09]
    Dist_Virt_Sat:              [42164.]
    Earth_Radius_Equator:       [6378.137]
    Earth_Radius_Polar:         [6356.7523]
    LFAC:                       [20466275]
    LOFF:                       [2750.5]
    Nadir_latitude:             [-0.03231456]
    Nadir_longitude:            [140.2600616]
    Req2_info:                  [1.0067395]
    Req_info:                   [0.00669438]
    Rpol_info:                  [0.99330562]
    SSP_latitude:               [-0.00803651]
    SSP_longitude:              [140.66196304]
    Sub_Lon:                    [140.7]
    Sun_Pos:                    [-8.94166471e+07  1.12654081e+08  4.88327422e...
    destripe:                   no
    easternmost_longitude:      [-138.07843]
    northernmost_latitude:      [81.05099]
    southernmost_latitude:      [-81.05099]
    westernmost_longitude:      [59.47843]
    Conventions:                CF-1.7
    Metadata_Conventions:       Unidata Dataset Discovery v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              swath
    comment:                    This file contains sea surface temperature da...
    creator_email:              Alex.Ignatov@noaa.gov
    creator_name:               Alex Ignatov
    creator_url:                http://www.star.nesdis.noaa.gov
    gds_version_id:             02.0
    geospatial_lat_units:       degrees_north
    geospatial_lon_units:       degrees_east
    institution:                NOAA/NESDIS/STAR
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    GHRSST protocol describes data use as free an...
    naming_authority:           org.ghrsst
    netcdf_version_id:          4.6.3 of Apr  2 2019 11:58:26 $
    processing_level:           L2P
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_email:            ghrsst-po@

#### Now write this to our shared AWS S3 bucket

Note that in order to do this you need the aws command line tools which can be installed by running from the command line 

`pip install awscli`

`aws s3 sync ./goes_east s3://ohw-bucket/goes_east`

`aws s3 sync ./goes_west s3://ohw-bucket/goes_west`

`aws s3 sync ./goes_west s3://ohw-bucket/himawari`

#### note that putting the ! in front of a command in jupyter send it to the terminal so you could run it here with



In [ ]:
! pip install awscli
#! aws s3 sync ./goes_east s3://ohw-bucket/goes_east
#! aws s3 sync ./goes_west s3://ohw-bucket/goes_west
! aws s3 sync ./himawari s3://ohw-bucket/himawari

## Test reading the data

In [ ]:
%%time

sat = 'himawari'
file_location = 's3://ohw-bucket/'+sat

ds = xr.open_zarr(fsspec.get_mapper(file_location,anon=False))

ds